In [57]:
import pandas as pd
import os
import json

In [58]:
dirs_path = os.path.join('logs')
dirs = os.listdir(dirs_path)
# dirs = [f'exp{i}' for i in range(86, 92+1)]

In [59]:
all_results = pd.DataFrame()

In [60]:
def get_data_from_config(path, columns):
    path = os.path.join(path, 'grid_search')
    metrics = ['f1_weighted','matthews_corrcoef', 'balanced_accuracy']
    aux = pd.DataFrame()
    for file in os.listdir(path):
        df = pd.read_csv(os.path.join(path, file))
        data = pd.DataFrame()
        filename = file.split('_')[0]
        data['config'] = df['params']
        data['model'] = filename
        for metric in metrics:
            data[f'avg_{metric}'] = df[f'mean_test_{metric}']
            data[f'std_{metric}'] = df[f'std_test_{metric}']
        data = data[columns]
        sorted_results = data.sort_values(by=['avg_matthews_corrcoef', 'avg_f1_weighted'])
        sorted_results = sorted_results.iloc[:min(len(sorted_results), 5)]
        aux = pd.concat([aux, sorted_results], ignore_index=True)
    return aux
        

In [61]:
def get_data_from_metric(path):
    path = os.path.join(path, 'model_metrics.csv')
    if not os.path.exists(path):
        return pd.DataFrame()
    data = pd.read_csv(path)
    
    mean = data.groupby(['model','config']).mean()
    mean = mean.reset_index()

    std =  data.groupby(['model','config']).std()
    std = std.reset_index()
    metrics = ['f1_weighted','matthews_corrcoef', 'balanced_accuracy']

    new_data = pd.DataFrame()
    new_data['model'] = mean['model']
    new_data['config'] = mean['config']

    for i in metrics:
        new_data["avg_"+i] = mean[i]
        new_data["std_"+i] = std[i]

    return new_data

    

In [62]:
data = pd.DataFrame()
metrics = ['f1_weighted','matthews_corrcoef', 'balanced_accuracy']
columns = ['model', 'config']

for metric in metrics:
        columns.append(f'avg_{metric}')
        columns.append(f'std_{metric}')
print(columns)

['model', 'config', 'avg_f1_weighted', 'std_f1_weighted', 'avg_matthews_corrcoef', 'std_matthews_corrcoef', 'avg_balanced_accuracy', 'std_balanced_accuracy']


In [63]:
for d in dirs:
    path = os.path.join(dirs_path, d)
    print(d)
    if d == 'exp00':
        continue
    with open(os.path.join(path, 'config.json'), 'r') as file:
        config = json.load(file)
        if config.get('grid_search', False):
            df = get_data_from_config(path, columns)
        else:
            df = get_data_from_metric(path)
        if len(df) != 0:
            df['norm_model'] = config.get('norm_model', 'Standard')
            df['columns_to_drop'] = str(config.get('columns_to_drop', []))
            df['number_best_features'] = config.get('number_best_features', 'all')
            df['variance_threshold'] = config.get('variance_threshold', 0)
            df['balance_dataset'] = config.get('balance_dataset', 'null')
            df['exp'] = f'joana {d}'
            data = pd.concat([data, df], ignore_index=True)
        

exp00
exp01
exp02
exp03
exp04
exp05
exp06
exp07
exp08
exp09
exp10
exp41
exp51
exp62
exp63
exp64
exp65
exp66
exp67
exp68
exp69
exp70
exp71
exp72
exp73
exp74
exp75
exp76
exp77
exp78
exp79
exp80
exp83
exp84
exp85
exp86
exp87
exp88
exp89
exp90
exp91
exp92
exp93
exp94


In [64]:
data.to_csv('results_mean_std_lasts.csv', index=False)